# About the Atlantic Forest

The Atlantic Forest (Mata Atlântica; Bosque Atlántico) is the second largest rainforest in South America. It is located in the Atlantic coast of Brazil (92 percent of its total area), northern Argentina (2 percent of its total area), and southern Paraguay (6 percent of its total area). The forest is the third most extensive biome in Brazil after the Amazon and the Cerrado and is considered one of the most threatened forests in the world, with only about 12 percent of its original area remaining (de Lima Palidon and dos Santos Guapyassu, 2005).

The Atlantic Forest is highly fragmented and is pressured by urbanization, deforestation, and agricultural plantations (Singh and Huang, 2022). The original area of the forest covered 15 percent of the land area of Brazil. Now, it is estimated that only about 8.5 percent of its area surpasses 100 hectares in size (Burak et al., 2023). The Atlantic Forest’s ecoregion composes some of Brazil’s largest cities, including Rio de Janeiro and São Paulo. This region produces an estimated 70 percent of the country’s gross domestic product (World Wildlife Fund n.d). About 35 percent of the remaining area is a protected area with 14 national parks, 14 biological reserves, and 6 ecological stations (de Lima Palidon and dos Santos Guapyassu, 2005).

The area has high levels of biodiversity. An estimated 2,200 species of birds, mammals, reptiles and amphibians can be found in the Atlantic Forest (The Nature Conservancy n.d.). About 40 percent of its vascular plants and about 60 percent of its vertebrate species are endemic species that cannot be found anywhere else in the world. In Brazil, there are an estimated 20,000 species of plants in the Atlantic Forest (about 8 percent of the Earth’s total)(The Nature Conservancy n.d).

In [10]:
import io
import os
import zipfile
from pathlib import Path

import ee
import geemap
import geopandas as gpd
import requests

from utils.mapping import (
    create_geemap,
    forest_type_dict,
    matplotlib_ramp_to_hexcodes,
)

In [2]:
ee.Authenticate()
ee.Initialize()

# Boundaries

The boundary for the Atlantic Forest was sourced from the Spatial Ecology and Conservation Lab (LEEC) using the consensual limit corresponding to the intersection of the limits reviewed by the study.

Citation:
Muylaert, R.L.; Vancine, M.H.; Bernardo, R.; Oshima, J.E.F.; Sobral-Souza, T.; Tonetti, V.R.; Niebuhr, B.B.; Ribeiro, M.C. 2018. Uma nota sobre os limites territoriais da Mata Atlântica. Oecologia Australis 22: 3. DOI: 10.4257/oeco.2018.2203.09

In [3]:
boundary_url = url = (
    "https://github.com/LEEClab/ATLANTIC-limits/raw/master/data/limites_integradores_wgs84_v1_2_0.zip"
)
geometry_folder = Path("../../geometries")
boundary_name = "ma_limite_consensual_muylaert_et_al_2018_wgs84"
boundary_folder = geometry_folder / "limites_integradores_wgs84_v1_2_0/"

In [ ]:
response = requests.get(url)
if response.status_code == 200:
    with zipfile.ZipFile(io.BytesIO(response.content)) as z:
        z.extractall(geometry_folder)
    print(f"Files are saved in {geometry_folder}")
else:
    print("Status code:", response.status_code)

In [12]:
# Keep only consensus boundary

for file_name in os.listdir(boundary_folder):
    # Check if the file name does not contain the required substring
    if "ma_limite_consensual_muylaert_et_al_2018_wgs84" not in file_name:
        # Delete the file
        file_path = os.path.join(boundary_folder, file_name)
        os.remove(file_path)

In [4]:
af_gdf = gpd.read_file(
    f"{boundary_folder}/ma_limite_consensual_muylaert_et_al_2018_wgs84.shp"
)
af_fc = ee.FeatureCollection(af_gdf.__geo_interface__)

In [ ]:
min_reducer = ee.Reducer.min()
max_reducer = ee.Reducer.max()
combined_reducer = min_reducer.combine(reducer2=max_reducer, sharedInputs=True)

# Forest Type

The Copernicus Land Cover layer categorizes the Atlantic Forest as an evergreen broad leaf forest. The literature classifies different areas of the forest into coastal, deciduous and semideciduous inland forests, low montane forests, as well as ecosystems including mangrove swamps, and high-altitude swamps (de Lima Palidon and dos Santos Guapyassu, 2005). The coastal forest has been the most disturbed, with only about 3 percent of its original area remaining. 

In [7]:
forest_type = ee.Image("COPERNICUS/Landcover/100m/Proba-V-C3/Global/2019").select(
    "forest_type"
)
forest_type

In [8]:
forest_type_clip = forest_type.clip(af_fc)

In [9]:
forest_type_filtered = forest_type_clip.updateMask(forest_type.neq(0))

In [10]:
create_geemap(
    image=forest_type_filtered,
    data_viz=forest_type_dict,
    center_by=af_fc,
    map_title="Forest Type (Copernicus Land Cover)",
    categorical=True,
)

Map(center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(childr…

![Atlantic Forest Forest Type](../../maps/atlantic_forest_types.png)

In [99]:
land_cover_classification = ee.Image(
    "COPERNICUS/Landcover/100m/Proba-V-C3/Global/2019"
).select("discrete_classification")

In [100]:
land_cover_clipped = land_cover_classification.clip(af_fc)

In [101]:
p = geemap.Map()
p.centerObject(af_fc)
p.addLayer(land_cover_clipped, {}, "Copernicus Global Land Cover Layers")
# text = "Made by Ilse Paniagua"
# p.add_text(text)
# p.add_legend(title="", legend_dict=land_cover_dict)
p

Map(center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(childr…

# Forest Loss

The forest has been disturbed for over 500 years due to logging and clearing for the cultivation of sugarcane, coffee, and cacao plantations that drove exports in Brazil from early colonial times to early in the 20th century (de Lima Palidon and dos Santos Guapyassu, 2005). In recent history, forest disturbances have been driven by expanding human populations and land conversion to urban areas. Figure 4 shows the year of forest loss in the Atlantic Forest from 2001 to 2023. Coastal areas saw deforestation in the early 2000s, while inland and outer forest sections have seen deforestation in the 2010s and 2020s. 

In [89]:
hansen = ee.Image("UMD/hansen/global_forest_change_2023_v1_11").select("lossyear")
hansen_clipped = hansen.clip(af_fc)

In [94]:
loss_ramp = {
    "palette": "cividis",
    "min": 1,
    "max": 23,
}

In [92]:
loss_colors = matplotlib_ramp_to_hexcodes(color_ramp="cividis")

In [97]:
p = None
p = geemap.Map()
p.addLayer(hansen_clipped, loss_ramp, "Year of Forest Loss")
p.centerObject(af_fc)
# p.add_text("Made by Ilse Paniagua")
# p.add_colorbar_branca(colors=loss_colors, vmin=1, vmax=23, orientation="horizontal", caption="Year of Forest Loss (2000-2023)",)
p

Map(center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(childr…

![](../../maps/atlantic_forest_year_of_forest_loss.png)

# Carbon Density

The literature shows a diverse range in carbon density throughout the Atlantic Forest. Due to it fragmented location in high elevation areas, I expect carbon density to be lower in the Atlantic Forest compared to biomes like the Amazon. I also expect above ground biomass (AGB) to be higher than below ground biomass (BGB) throughout the forest.

I expect there to be some variation in ABG throughout the forest. AGB is likely higher in the inland forest section compared to the coastal forest. Soil in the coastal forest has been described as sandy, infertile soil, which likely affects the composition of trees due to poor soil quality (de Lima Palidon and dos Santos Guapyassu, 2005).

The figure below shows AGB carbon estimates from the Global Land Analysis and Disccovery at the University of Maryland using Global Ecosystem Dynamics Investigation (GEDI) lidar data and Landsat time series data. Estimates are provided for 2019 at a 30-meter spatial resolution (Potapov et al., 2020). The maximum canopy height estimated in the Atlantic Forest is about 30 meters.


In [35]:
canopy_height_collection = ee.ImageCollection("users/potapovpeter/GEDI_V27")

In [45]:
canopy_height_clip = canopy_height_collection.median().clip(af_fc)

In [ ]:
canopy_height_stats = canopy_height_clip.reduceRegion(
    reducer=combined_reducer, geometry=af_fc, scale=30, bestEffort=True
)
print(canopy_height_stats.getInfo())

In [75]:
canopy_viz = {"palette": "YlGnBu", "min": 0, "max": 30}

In [76]:
ylgnbl_hexcodes = matplotlib_ramp_to_hexcodes(color_ramp="YlGnBu", n_colors=9)

In [77]:
w = geemap.Map()
w.addLayer(canopy_height_clip, canopy_viz, "Canopy Height (GEDI)")
w.centerObject(af_fc)
# w.add_colorbar_branca(
#     colors=ylgnbl_hexcodes,
#     vmin=5,
#     vmax=25,
#     layer_name="Canopy Height (GEDI)",
#     orientation="horizontal",
#     caption="Canopy Height (m)",
# )
w.add_text("Made by Ilse Paniagua")
w

Map(center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(childr…

In [78]:
roi = w.user_roi

In [ ]:
roi.getInfo()

In [84]:
polygon = ee.Geometry.Polygon(
    [
        [
            [-60.890352, -30.501934],
            [-60.890352, -12.271573],
            [-33.336884, -12.271573],
            [-33.336884, -30.501934],
            [-60.890352, -30.501934],
        ]
    ]
)

polygon_feature = ee.Feature(polygon)
roi_fc = ee.FeatureCollection([polygon_feature])

In [88]:
s = geemap.Map()
s.addLayer(
    canopy_height_clip,
    {"palette": "YlGnBu", "min": 5, "max": 20},
    "Canopy Height (GEDI)",
)
s.centerObject(roi_fc)
s

Map(center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(childr…

![](../../maps/atlantic_forest_canopy_height.png)

# Elevation

There is a diversity in elevation and topographies in the Atlantic Forest. Elevation ranges from around -100 to 2800 meters in altitude, with distinct microclimates observed with differences in slope of about 100 meters (Burak et al., 2023). Although typically carbon density decreases with elevation, I believe carbon density likely increases with elevation in sections of the Atlantic Forest due to the historic role that elevation has played in which areas get deforested or degraded. For example, Hengl et al. (2015) found that both above and below ground biomass increased with elevation. A confounding element in this study is that remaining sections of the forest are largely inaccessible and cannot be cost-effectively cleared due to high elevations. This gives the impression that AGB increases with elevation, ignoring that forests in a lower elevation have already been converted to other land uses but might have a higher AGB in a counterfactual scenario.

The figure below shows a comparison of elevation and canopy height estimates. The box shows a peak in elevation near the metropolitan city of Rio de Janeiro accompanied by high canopy heights.


In [26]:
dem = ee.ImageCollection("COPERNICUS/DEM/GLO30")
elevation = dem.select("DEM").median()
elevation_clipped = elevation.clipToCollection(af_fc)

In [42]:
# Apply the reduceRegion method to get the minimum value.
elevation_stats = elevation_clipped.reduceRegion(
    reducer=combined_reducer, geometry=af_fc, scale=30, bestEffort=True
)

In [43]:
print(elevation_stats.getInfo())

{'DEM_max': 2807.846435546875, 'DEM_min': -102.14678955078125}


In [ ]:
terrain_colors = matplotlib_ramp_to_hexcodes(color_ramp="terrain")

In [17]:
elevation_viz = {
    "min": -100,
    "max": 2800.0,
    "palette": terrain_colors,
}

In [67]:
q = geemap.Map()
q.centerObject(af_fc)
q.addLayer(elevation_clipped, elevation_viz, "Elevation (m)")
q.add_colorbar_branca(
    colors=terrain_colors,
    vmin=0,
    vmax=2000,
    layer_name="Elevation (m)",
    caption="Elevation (m)",
)
q.add_text("Made by Ilse Paniagua")
q

Map(center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(childr…

In [87]:
t = geemap.Map()
t.addLayer(elevation_clipped, elevation_viz)
t.centerObject(roi_fc)
t

Map(center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(childr…

![](../../maps/atlantic_forest_elevation_canpopy_comparison.png)

# 